In [1]:
!pip install fhirpathpy
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from fhirpathpy import evaluate

 Notice there's no value[x] for Systolic blood pressure.

In [3]:
obs = {
  "resourceType": "Observation",
  "id": "example-missing",
  "status": "final",
  "code": {
    "text": "Blood Pressure"
  },
  "component": [
    {
      "code": {
        "text": "Systolic blood pressure"
      }
    },
    {
      "code": {
        "text": "Diastolic blood pressure"
      },
      "valueQuantity": {
        "value": 80,
        "unit": "mmHg"
      }
    }
  ]
}


In [5]:
result = evaluate(obs, "component.valueQuantity.value > 70", [])
print(result)

[True]


In [10]:
result = evaluate(obs, "component.where(valueQuantity.exists()).valueQuantity.value.where($this > 70).exists()", [])
print(result)

[True]


In [8]:
result = evaluate(obs, "component.where(code.text = 'Systolic blood pressure').valueQuantity.value > 120", [])
print(result)

[]


Notice the correct “FHIRPath logic” is to chain two .where(...) calls (or a single .where() with a combined condition) on the component collection, rather than calling .exists() mid-chain.

In [16]:
result = evaluate(obs, "component.where(code.text = 'Systolic blood pressure').where(valueQuantity.exists())", [])
print(result)

[]


Debugging Tips
Check the Raw Data: Print or inspect the obs structure in your notebook to confirm the spelling of component and the actual numeric values.
Test Intermediate Steps:
component.valueQuantity.value

In [11]:
result = evaluate(obs, "component.valueQuantity.value", [])
print(result)

[80]


Below is an example FHIR Observation where two component entries use different data types for value[x]: one uses valueQuantity, and the other uses valueCodeableConcept. This mix can cause type mismatches if you try to perform numeric operations on the entire collection without first filtering for numeric types.



obs2 = {
  "resourceType": "Observation",
  "id": "mixed-type-example",
  "status": "final",
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "85354-9",
        "display": "Blood pressure panel"
      }
    ],
    "text": "Blood Pressure"
  },
  "component": [
    {
      "code": {
        "coding": [
          {
            "system": "http://loinc.org",
            "code": "8480-6",
            "display": "Systolic blood pressure"
          }
        ]
      },
      "valueQuantity": {
        "value": 120,
        "unit": "mmHg"
      }
    },
    {
      "code": {
        "coding": [
          {
            "system": "http://snomed.info/sct",
            "code": "373067005",
            "display": "Diagnosis of Hypertension"
          }
        ],
        "text": "Diagnostic Reason"
      },
      "valueCodeableConcept": {
        "coding": [
          {
            "system": "http://snomed.info/sct",
            "code": "38341003",
            "display": "Hypertension"
          }
        ],
        "text": "Hypertensive Disorder"
      }
    }
  ]
}


In [24]:
result = evaluate(obs, "component.valueQuantity.value > 140", [])
print(result)

[False]


In [23]:
result = evaluate(obs, "component.valueQuantity.value", [])
print(result)

[80]


In [ ]:
pt = {
  "resourceType": "Patient",
  "id": "fake-patient-example",
  "name": [
    {
      "use": "official",
      "family": "Doe",
      "given": ["Jane", "Elizabeth"]
    }
  ],
  "address": [
    {
      "use": "home",
      "line": ["1234 Main Street"],
      "city": "Exampleville",
      "state": "CA",
      "postalCode": "90001",
      "country": "USA"
    }
  ],
  "birthDate": "1980-01-15"
}
